In [ ]:
1.简述OS中断的作用
    中断装置由一些特定的寄存器和控制线路组成，中央处理器和外围设备等识别到的
事件保存在特定的寄存器中。
    中央处理器每执行完一条指令，均由中断装置判别是否有事件发生。
    若无事件发生，CPU继续执行；
    若有事件发生，则中断装置中断原占有CPU的程序的执行，让操作系统的处理事件
服务程序占用CPU，对出现的事件进行处理，事件处理完后，再让原来的程序继续占用CPU执行


2.内核态与用户态
     内核态：cpu可以访问内存的所有数据，包括外围设备，
例如硬盘，网卡，cpu也可以将自己从一个程序切换到另一个程序。
 
    用户态：只能受限的访问内存，且不允许访问外围设备，占用cpu的能力被剥夺，
cpu资源可以被其他程序获取

    由于需要限制不同的程序之间的访问能力，防止他们获取别的程序的内存数据，
或者获取外围设备的数据，并发送到网络，cpu划分出两个权限等级：用户态和内核态。


3.进程、线程、系统调用
    进程：正在执行的一个程序或者说一个任务，负责执行任务的是cpu。
进程是用来把资源集中到一起的，进程是资源单位，或者说资源集合。

    线程：线程是cpu上的执行单位。同一个进程内的多个线程共享该进程内的地址资源。
线程是操作系统能够进行运算调度的最小单位。

    系统调用： 所有用户程序都是运行在用户态的，但是有时候程序确实需要做一些内核态的事情，
例如从硬盘读取数据，或者从键盘获取输入等，
    而唯一能做这些事情的就是操作系统，所以此时程序就需要向操作系统请求以程序
的名义来执行这些操作。这时，就需要一个机制：用户态程序切换到内核态，但是不能
控制在内核态中执行的指令。这种机制就叫系统调用。

In [ ]:
4.threading module中event/condition的区别：
    condition: 某些事件触发或达到特定的条件后才处理数据，默认创建了一个lock对象。
        con = threading.Condition()
        con.acquire()
        con.notify()
        con.wait()
        con.release()

    event：其他线程需要通过判断某个线程的状态来确定自己的下一步操作，就可以用event。
        from threading import Event
        event = Event()
        event.set()： 设置event的状态值为True，所有阻塞池的线程激活进入就绪状态, 等待操作系统调度；
        event.is_set()：返回event的状态值；
        event.wait()：如果 event.is_set()==False将阻塞线程；
        event.clear()：恢复event的状态值为False。

In [ ]:
5.进程间的通信方式：消息队列、管道、信号量、信号、共享内存、套接字

    管道：是两个进程间进行单项通信机制，读时阻塞写，写时阻塞读 -> 半双工管道
    
    threading.Semaphore(value=-1)：为1时，表示只有一个线程能够拿到许可，
    其他线程都处于阻塞状态，直到该线程释放为止。当然信号量不可能永久的阻塞在那里。信号量也提供了超时处理机制。
    如果传入了 -1，则表示无限期的等待。

6.管道 VS 队列： 队列= 管道 + Lock
    from multiprocessing import Queue,Process
        queue = Queue()
        queue.put(url)
        url = queue.get()
    from multiprocessing import Pipe,Process
        pipe = Pipe()
        pipe.send(url)
        pipe.recv()
        
        
7.IO多路复用：
    IO多路复用实际上就是用select，poll,epoll监听多个io对象，当io对象有
变化（有数据）的时候就通知用户进程。好处就是单个进程可以处理多个socket。
 
    1.当用户进程调用了select，那么整个进程会被block；

    2.而同时，kernel会“监视”所有select负责的socket；

    3.当任何一个socket中的数据准备好了，select就会返回；

    4.这个时候用户进程再调用read操作，将数据从kernel拷贝到用户进程。

    总结：
        1.I/O 多路复用的特点是通过一种机制一个进程能同时等待多个文件描述符，
    而这些文件描述符（套接字描述符）其中的任意一个进入读就绪状态，
    select()函数就可以返回。

        2.IO多路复用：需要两个系统调用，system call (select 和 recvfrom)，
    而blocking IO只调用了一个system call (recvfrom)。但是，用select的优势
    在于它可以同时处理多个connection。

        3.如果处理的连接数不是很高的话，使用select/epoll的web server不一定
    比使用多线程 + 阻塞 IO的web server性能更好，可能延迟还更大。

        4.select/epoll的优势并不是对于单个连接能处理得更快，而是在于能处理更多的连接。
        
        
8.同步IO、异步IO
    synchronous io: 做”IO operation”的时候会将process阻塞;”IO operation”是指真实的IO操作
                blocking IO，non-blocking IO，IO multiplexing都属于synchronous IO这一类.
 
    asynchronous io: 当进程发起IO 操作之后，就直接返回再也不理睬了，直到kernel发送一个信号，
                 告诉进程说IO完成。在这整个过程中，进程完全没有被block。
    
    异步io的实现会负责把数据从内核拷贝到用户空间。
    
    

In [ ]:
9.线程、协程
    协程：单线程下的并发。协程是一种用户态的轻量级线程，即协程是由用户程序自己控制调度的。
 
        1.python的线程是属于内核级别的，即由操作系统控制调度（如单线程
    遇到io或执行时间过长就会被迫交出cpu执行权限，切换其他的线程运行）

        2.单线程内开启协程，一旦遇到io，就会从应用程序级别（而非操作系统）
    控制切换，以此来提升效率（！！非io操作的切换与效率无关）
    
        3.协程指的是单个线程，因而一旦协程出现阻塞，将会阻塞整个线程。

In [4]:
# ——*—— 写一个包含10个线程的程序，主线程必须等待每一个子线程执行完成之后才结束执行，每一个子线程执行的时候都需要打印当前线程名、当前活跃线程数量 ——*—— #
from threading import Thread, currentThread, activeCount
import time

def task(n):
    print('Thread %s : %s' %(currentThread().name, n))
    time.sleep(1)
    print('Active num: ',activeCount())

threads = []
for i in range(10):
    t = Thread(target=task,args=('james_%s'%i,))
    t.start()
    threads.append(t)
    
for i in threads:
    i.join()
    
print('Process End.')

Thread Thread-26 : james_0Thread Thread-27 : james_1

Thread Thread-28 : james_2Thread Thread-29 : james_3

Thread Thread-30 : james_4
Thread Thread-31 : james_5
Thread Thread-32 : james_6Thread Thread-33 : james_7

Thread Thread-34 : james_8
Thread Thread-35 : james_9
Active num: Active num:  15 15

Active num: Active num:  13
 13
Active num:  11
Active num:  10
Active num:  9
Active num: Active num:   88

Active num:  6
Process End.


In [7]:
# _*_ 使用协程创建一个 producer/consumer module _*_ #
def consumer():
    while True:
        x = yield #无参的时候，可以用这种方式以传入值作为参数
        print('Consume :',x)
        
def producer():
    c = consumer()
    next(c)
    for i in range(10):
        print('Product :',i)
        c.send(i)
        
producer()

Product : 0
Consume : 0
Product : 1
Consume : 1
Product : 2
Consume : 2
Product : 3
Consume : 3
Product : 4
Consume : 4
Product : 5
Consume : 5
Product : 6
Consume : 6
Product : 7
Consume : 7
Product : 8
Consume : 8
Product : 9
Consume : 9


In [11]:
# ——*—— 写一个程序，包含十个线程，子线程必须等待主线程sleep 10秒钟之后才执行，并打印当前时间 ——*—— #
from threading import Thread, Event
import datetime

def task():
    event.wait(10)
    print('Time:',datetime.datetime.now())
    
event = Event()
for i in range(10):
    t = Thread(target=task)
    t.start()
    
time.sleep(10)
event.set()

Time: 2021-07-02 14:12:59.631592
Time:Time:Time:Time: 2021-07-02 14:12:59.691562
 2021-07-02 14:12:59.666540
Time: 2021-07-02 14:12:59.741537
Time: Time:Time: 2021-07-02 14:12:59.791474
 Time: 2021-07-02 14:12:59.841447
2021-07-02 14:12:59.7664912021-07-02 14:12:59.716546
  2021-07-02 14:12:59.821470
2021-07-02 14:12:59.861471



In [12]:
# _*_ 信号量实现：十个线程，最多有五个可以并行执行 _*_ #
from threading import Thread, Semaphore, currentThread
import time
def task(n):
    sm.acquire()
    print('Thread start: %s -> %s'%(currentThread().name,n))
    time.sleep(1)
    print('Thread end')
    sm.release()

for i in range(10):
    sm = Semaphore(5)
    t = Thread(target=task, args=(i,))
    t.start()
    

Thread start: Thread-66 -> 0
Thread start: Thread-67 -> 1Thread start: Thread-68 -> 2

Thread start: Thread-69 -> 3Thread start: Thread-70 -> 4

Thread start: Thread-71 -> 5Thread start: Thread-72 -> 6

Thread start: Thread-73 -> 7Thread start: Thread-74 -> 8

Thread start: Thread-75 -> 9
Thread end
Thread endThread end

Thread endThread end

Thread endThread end

Thread endThread end

Thread end


In [13]:
# _*_ 不使用time.sleep()实现：主程序运行5s后，执行特定函数 _*_ #
from threading import Timer
def hello(name):
    print('%s say %s'%(name,'hello world!'))
    
t = Timer(5, hello, args=('james',))
t.start()

james say hello world!


In [16]:
# _*_ 使用Queue实现进程间通信 _*_ #
from multiprocessing import Process,Queue,current_process,JoinableQueue
import time

def consumer(q):
    while True:
        res = q.get()
        if not res: break
        print('Consume: ',res,'--',current_process().name)
        
def producter(q):
    for i in range(8):
        print('Produce: ',i)
        time.sleep(1)
        q.put(i)
        
q = Queue()
p1 = Process(target=producter, args=(q,))
c1 = Process(target=consumer, args=(q,))
c2 = Process(target=consumer, args=(q,))
p1.start()
c1.start()
c2.start()
p1.join()
q.put(None)
q.put(None)
print("Process End")

# Joinable Queue
def consumer(q):
    while True:
        res = q.get()
        if not res: break
        print('Consume: ',res,'--',current_process().name)
        q.task_done()
        
def producter(q):
    for i in range(8):
        print('Produce: ',i)
        time.sleep(1)
        q.put(i)
    q.join()
    
q = JoinableQueue()
p1 = Process(target=producter, args=(q,))
p2 = Process(target=producter, args=(q,))
c1 = Process(target=consumer, args=(q,))
c2 = Process(target=consumer, args=(q,))
p1.start()
p2.start()

c1.daemon = True
c2.daemon = True
c1.start()
c2.start()
p1.join()
p2.join()
print('Process End')

Process End
Process End


In [ ]:
# _*_ Pipe 实现进程间通信 _*_ #
from multiprocessing import Process,Pipe

def task(conn):
    conn.send('hello world')
    conn.close()
    
parent_conn, child_conn = Pipe()
p = Process(target=task, args=(child_conn,))
p.start()
p.join()
print(parent_conn.recv())

In [ ]:
# _*_ 使用selectors模块创建一个处理客户端消息的服务器程序 _*_ #
# server  blocking IO 
import socket
server = socket.socket(socket.AF_INET,socket.SOCK_STREAM)
server.bind(('127.0.0.1',8080))
server.listen(5)
while True:
    conn,addr = server.accept()
    print(addr)
    while True:
        try:
            data = conn.recv(1024)
            if not data: break
            conn.send(data.upper())
        except Exception as e:
            print(e)
            break
             
# server  IO多路复用 selectors 会根据操作系统选择select poll epoll          
import socket
import selectors
sel = selectors.DefaultSelector()
def accept(server_fileobj,mask):
    conn,addr = server_fileobj.accept()
    print(addr)
    sel.register(conn,selectors.EVENT_READ,read)
     
def read(conn,mask):
    try:
        data = conn.recv(1024)
        if not data:
            print('closing..',conn)
            sel.unregister(conn)
            conn.close()
            return
        conn.send(data.upper())
    except Exception:
        print('closeing...',conn)
        sel.unregister(conn)
        conn.close()
         
server_fileobj = socket.socket(socket.AF_INET,socket.SOCK_STREAM)
server_fileobj.bind(('127.0.0.1',8080))
server_fileobj.listen(5)
server_fileobj.setblocking(False)
sel.register(server_fileobj,selectors.EVENT_READ,accept)
while True:
    events = sel.select()
    for sel_obj,mask in events:
        callback = sel_obj.data
        callback(sel_obj.fileobj,mask)
         
# client
# -*- coding:utf-8 -*-
import socket
client = socket.socket(socket.AF_INET,socket.SOCK_STREAM)
client.connect(('127.0.0.1',8080))
while True:
    msg = input('>>>:').strip()
    if not msg:continue
    client.send(msg.encode('utf-8'))
    data = client.recv(1024)
    print(data.decode('utf-8'))

In [ ]:
# _*_ 使用socketserver模块，实现一个支持同时处理多个客户端请求的服务器，要求每次启动一个新线程处理客户端请求 _*_ #
'''
socketserver模块类介绍
      SocketServer内部使用 IO多路复用 以及 “多线程” 和 “多进程” ，
从而实现并发处理多个客户端请求的Socket服务端。即：每个客户端请求
连接到服务器时，Socket服务端都会在服务器是创建一个“线程”或者“进 程”
 专门负责处理当前客户端的所有请求。
        socketserver模块可以简化网络服务器的编写，python把网络服务抽
象成两个主要的类，一个是server类，用于处理连接相关的网络操作，另一个
是RequestHandler类，用于处理数据相关的操作。并且提供两个Mixln类，
用于扩展server，实现多进程或者多线程。
'''
import socketserver
import threading

class MyServer(socketserver.BaseRequestHandler):
    def handle(self):
        while True:
            self.data = self.request.recv(1024).decode()
            print(self.data)
            self.request.send(self.data.upper().encode())
            
server = socketserver.ThreadingTCPServer(('127.0.0.1',9999), MyServer)
t = threading.Thread(target=server.server_forever)
t.start()